In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
#import preprocessor as p
import re
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import string
import html
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import os
import pickle
from sklearn.metrics import classification_report, accuracy_score
from google.colab import files
import io

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install preprocessor
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4473 sha256=890f370b7f1aff0f3c19c9ff9cdb736bf9b8ab6b6785fd6882fed88158df0505
  Stored in directory: /root/.cache/pip/wheels/18/41/3e/30e0c08b0759166f31aee174d58d288ad5c277bd102c8ad869
Successfully built preprocessor
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requireme

In [3]:
uploaded = files.upload()
df_zelensky_may = pd.read_csv(io.BytesIO(uploaded['tweet_zelensky_may.csv']))

Saving tweet_zelensky_may.csv to tweet_zelensky_may.csv


In [4]:
uploaded = files.upload()
df_zelensky_jun = pd.read_csv(io.BytesIO(uploaded['tweet_zelensky_jun.csv']))

Saving tweet_zelensky_jun.csv to tweet_zelensky_jun.csv


In [5]:
uploaded = files.upload()
df_zelensky_jul = pd.read_csv(io.BytesIO(uploaded['tweet_zelensky_jul.csv']))

Saving tweet_zelensky_jul.csv to tweet_zelensky_jul.csv


In [8]:
col = ["Unnamed:0", "ID", "Date", "Tweet"]
df_zelensky = pd.DataFrame(columns=col)

df_zelensky = pd.concat([df_zelensky, df_zelensky_may])
df_zelensky = pd.concat([df_zelensky, df_zelensky_jun])
df_zelensky = pd.concat([df_zelensky, df_zelensky_jul])

In [9]:
def clean_text(_sentence):

    #removes emojis, pictographs, map symbols and flags
    emoji_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F" 
          u"\U0001F300-\U0001F5FF" 
          u"\U0001F680-\U0001F6FF" 
          u"\U0001F1E0-\U0001F1FF" 
                            "]+", flags = re.UNICODE)
    _sentence = emoji_pattern.sub(r'',_sentence)

    #emoves tags and attributes
    _sentence = re.sub('(<[^>]*>)','',_sentence)

    #converts character code to string
    _sentence = html.unescape(_sentence)

    #removes urls
    _sentence = re.sub("(\w+:\/\/\S+)|(http[^\s]+)|(www.[^\s]+)",'',_sentence, flags=re.MULTILINE)
    
    #removes RT
    _sentence = re.sub("RT",'',_sentence)
    
    #converts the sentence to a lower case
    _sentence = _sentence.lower()

    #removes punctuation
    _sentence = re.sub(r'\@\w+|\#\w+','', _sentence)
    _sentence = _sentence.translate(str.maketrans('', '', (string.punctuation)))
    
    #checks for stop words
    stop_words = set(stopwords.words('english'))
    _sentence = [word for word in str(_sentence).split() if word not in stop_words]

    #joins the word and returns a sentence
    return " ".join(_sentence)

#function to preprocess tweets
def preprocess_tweets(tweets_df,column):

    #apply clean text in a vectorized format
    vfunc_clean_text = np.vectorize(clean_text)
    tweets_df[column] = vfunc_clean_text(tweets_df[column]) #create a new column that consists of preprocessed tweets
    
    return tweets_df

In [10]:
data_df_zelensky = preprocess_tweets(df_zelensky,'Tweet')

In [11]:
def Vader_process(data_df):
    sid = SentimentIntensityAnalyzer()
    res = [*data_df['Tweet'].apply(sid.polarity_scores)]
    #print(res[:3])

    sentiment_df = pd.DataFrame.from_records(res)
    data_df = pd.concat([data_df.reset_index(drop=True), sentiment_df.reset_index(drop=True)], axis=1, join="inner")
    #print(data_df.head())
    
    conditions = [
    (data_df['compound'] <= -0.5),
    (data_df['compound'] > -0.5) & (data_df['compound'] < 0.5),
    (data_df['compound'] >= 0.5)
    ]

    values = [0, -1, 1]
    data_df['label'] = np.select(conditions, values)

    #print(data_df.head())
    
    X_df = data_df[data_df.label != -1]
    X_df = X_df[['Tweet','label']]
    #print(X_df.head())
    
    return X_df

In [12]:
X_zelensky = Vader_process(data_df_zelensky)

In [13]:
X_zelensky

,Tweet,label
1,russia ukraine war update live vladimir putin ...,0
4,ukraine’s president message australia tomorrow...,0
5,volodymyr zelensky says attempt ‘to humiliate ...,0
6,ukraine grateful poland people strong support ...,1
8,following fine praising hitler sexual assault ...,0
...,...,...
505989,whatever zelensky isnt stands way putins expan...,1
505991,pure eye peeing ruzzian spy aborted operation ...,0
505995,thats strange inaccurate assumption interestin...,1
505996,ukraine war zelenskyy orders mandatory evacuat...,0


**BERTopic Modelling**

In [14]:
import nltk
from nltk.corpus import stopwords
from umap import UMAP
from bertopic import BERTopic

In [15]:
umap_model = UMAP(random_state=3231)
bert_topic_model= BERTopic(embedding_model = "all-MiniLM-L6-v2", umap_model=umap_model)

bert_negative = X_zelensky[X_zelensky['label']==0] #change this to 1 if positive. keep 0 if negative.
bert_negative = bert_negative.Tweet.tolist()
topics, probs = bert_topic_model.fit_transform(bert_negative)

In [16]:
pd.set_option('display.max_rows', None)


In [17]:
topic_list=bert_topic_model.get_topic_info()
topic_list

,Topic,Count,Name
0,-1,50211,-1_nato_putin_dont_west
1,0,1702,0_money_laundering_billion_dollars
2,1,1472,1_impeached_trump_impeachment_extort
3,2,941,2_croatia_dejevsky_bahati_mary
4,3,786,3_biden_joe_djt_bidens
5,4,712,4_johnson_boris_johnsons_tory
6,5,571,5_minsk_implement_accords_agreements
7,6,556,6_azov_azovstal_battalion_fighters
8,7,514,7_jewish_jews_jew_holocaust
9,8,417,8_mariupol_steel_azovstal_plant


In [18]:
bert_topic_model.visualize_barchart()

In [19]:
#Positive
umap_model = UMAP(random_state=3231)
bert_topic_model= BERTopic(embedding_model = "all-MiniLM-L6-v2", umap_model=umap_model)

bert_positive = X_zelensky[X_zelensky['label']==1] #change this to 1 if positive. keep 0 if negative.
bert_positive = bert_positive.Tweet.tolist()
topics, probs = bert_topic_model.fit_transform(bert_positive)

pd.set_option('display.max_rows', None)
topic_list=bert_topic_model.get_topic_info()
topic_list

bert_topic_model.visualize_barchart()